##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Keras 和 Tensorflow Hub 對電影評論進行文本分類

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.org/tutorials/keras/text_classification_with_hub"><img src="https://tensorflow.org/images/tf_logo_32px.png" />在 tensorflow.org 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/zh-tw/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中運行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/zh-tw/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上查看源代碼</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/zh-tw/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.org/images/download_logo_32px.png" />下載 notebook</a>
  </td>
</table>

Note: 我們的 TensorFlow 社區翻譯了這些文檔。因為社區翻譯是盡力而為， 所以無法保證它們是最準確的，並且反映了最新的
[官方英文文檔](https://www.tensorflow.org/?hl=en)。如果您有改進此翻譯的建議， 請提交 pull request 到
[tensorflow/docs](https://github.com/tensorflow/docs) GitHub 倉庫。要志願地撰寫或者審核譯文，請加入
[docs@tensorflow.org Google Group](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs)。


此筆記本（notebook）使用評論文本將影評分為*積極（positive）*或*消極（nagetive）*兩類。這是一個*二元（binary）*或者二分類問題，一種重要且應用廣泛的機器學習問題。

本教程演示了使用 Tensorflow Hub 和 Keras 進行遷移學習的基本應用。

我們將使用來源於[網絡電影數據庫（Internet Movie Database）](https://www.imdb.com/)的 [IMDB 數據集（IMDB dataset）](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)，其包含 50,000 條影評文本。從該數據集切割出的 25,000 條評論用作訓練，另外 25,000 條用作測試。訓練集與測試集是*平衡的（balanced）*，意味著它們包含相等數量的積極和消極評論。

此筆記本（notebook）使用了[tf.keras](https://www.tensorflow.org/guide/keras)，它是一個 Tensorflow 中用於構建和訓練模型的高級API，此外還使用了 [TensorFlow Hub](https://www.tensorflow.org/hub)，一個用於遷移學習的庫和平台。有關使用 `tf.keras` 進行文本分類的更高級教程，請參閱 [MLCC文本分類指南（MLCC Text Classification Guide）](https://developers.google.com/machine-learning/guides/text-classification/)。

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## 下載 IMDB 數據集
IMDB數據集可以在 [Tensorflow 數據集](https://github.com/tensorflow/datasets)處獲取。以下代碼將 IMDB 數據集下載至您的機器（或 colab 運行時環境）中：

In [0]:
# 將訓練集按照 6:4 的比例進行切割，從而最終我們將得到 15,000
# 個訓練樣本, 10,000 個驗證樣本以及 25,000 個測試樣本
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

## 探索數據

讓我們花一點時間來了解數據的格式。每一個樣本都是一個表示電影評論和相應標籤的句子。該句子不以任何方式進行預處理。標籤是一個值為 0 或 1 的整數，其中 0 代表消極評論，1 代表積極評論。

我們來打印下前十個樣本。

In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

我們再打印下前十個標籤。

In [0]:
train_labels_batch

## 构建模型

神經網絡由堆疊的層來構建，這需要從三個主要方面來進行體系結構決策：

* 如何表示文本？
* 模型裡有多少層？
* 每個層裡有多少*隱層單元（hidden units）*？

本示例中，輸入數據由句子組成。預測的標籤為 0 或 1。

表示文本的一種方式是將句子轉換為嵌入向量（embeddings vectors）。我們可以使用一個預先訓練好的文本嵌入（text embedding）作為首層，這將具有三個優點：

* 我們不必擔心文本預處理
* 我們可以從遷移學習中受益
* 嵌入具有固定長度，更易於處理

針對此示例我們將使用 [TensorFlow Hub](https://www.tensorflow.org/hub) 中名為 [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) 的一種**預訓練文本嵌入（text embedding）模型** 。

為了達到本教程的目的還有其他三種預訓練模型可供測試：

* [google/tf2-preview/gnews-swivel-20dim-with-oov/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) ——類似 [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)，但 2.5%的詞彙轉換為未登錄詞桶（OOV buckets）。如果任務的詞彙與模型的詞彙沒有完全重疊，這將會有所幫助。
* [google/tf2-preview/nnlm-en-dim50/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1) ——一個擁有約 1M 詞彙量且維度為 50 的更大的模型。
* [google/tf2-preview/nnlm-en-dim128/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1) ——擁有約 1M 詞彙量且維度為128的更大的模型。

讓我們首先創建一個使用 Tensorflow Hub 模型嵌入（embed）語句的Keras層，並在幾個輸入樣本中進行嘗試。請注意無論輸入文本的長度如何，嵌入（embeddings）輸出的形狀都是：`(num_examples, embedding_dimension)`。


In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

現在讓我們構建完整模型：

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

層按順序堆疊以構建分類器：

1. 第一層是 Tensorflow Hub 層。這一層使用一個預訓練的保存好的模型來將句子映射為嵌入向量（embedding vector）。我們所使用的預訓練文本嵌入（embedding）模型([google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1))將句子切割為符號，嵌入（embed）每個符號然後進行合併。最終得到的維度是：`(num_examples, embedding_dimension)`。
2. 該定長輸出向量通過一個有 16 個隱層單元的全連接層（`Dense`）進行管道傳輸。
3. 最後一層與單個輸出結點緊密相連。使用 `Sigmoid` 激活函數，其函數值為介於 0 與 1 之間的浮點數，表示概率或置信水平。

讓我們編譯模型。

### 損失函數與優化器

一個模型需要損失函數和優化器來進行訓練。由於這是一個二分類問題且模型輸出概率值（一個使用 sigmoid 激活函數的單一單元層），我們將使用 `binary_crossentropy` 損失函數。

這不是損失函數的唯一選擇，例如，您可以選擇 `mean_squared_error` 。但是，一般來說 `binary_crossentropy` 更適合處理概率——它能夠度量概率分佈之間的“距離”，或者在我們的示例中，指的是度量 ground-truth 分佈與預測值之間的“距離”。

稍後，當我們研究回歸問題（例如，預測房價）時，我們將介紹如何使用另一種叫做均方誤差的損失函數。

現在，配置模型來使用優化器和損失函數：

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 訓練模型

以 512 個樣本的 mini-batch 大小迭代 20 個 epoch 來訓練模型。這是指對 `x_train` 和 `y_train` 張量中所有樣本的的 20 次迭代。在訓練過程中，監測來自驗證集的 10,000 個樣本上的損失值（loss）和準確率（accuracy）：

In [0]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## 評估模型

我們來看下模型的表現如何。將返回兩個值。損失值（loss）（一個表示誤差的數字，值越低越好）與準確率（accuracy）。

In [0]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

這種十分樸素的方法得到了約 87% 的準確率（accuracy）。若採用更好的方法，模型的準確率應當接近 95%。

## 進一步閱讀

有關使用字符串輸入的更一般方法，以及對訓練期間準確率（accuracy）和損失值（loss）更詳細的分析，請參閱[此處](https://www.tensorflow.org/tutorials/keras/basic_text_classification)。